In [1]:
import json
import pandas as pd
from pandas import json_normalize
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [55]:
df = pd.read_excel(r"C:\Users\Ragavan lingam\OneDrive\Desktop\MD112\Capstone\Car Dheko - Used Car Price Prediction\bangalore_cars.xlsx")

In [56]:
df['new_car_detail'] = df['new_car_detail'].apply(lambda x: json.loads(x.replace("'", '"').replace("None", "1")))
detail_df=pd.DataFrame(df.new_car_detail.to_list())
detail_df = detail_df.drop(['priceActual','priceSaving','priceFixedText','trendingText','oem','it','owner'],axis=1)
detail_df['price'] = detail_df['price'].apply(convert_price)
detail_df['City'] = 'Bangalore'
detail_df['SerialNumber'] = range(1, 1 + len(df))

In [2]:
def convert_price(price):
    price = re.sub(r'[₹]', '', price)
    if 'Lakh' in price:
        price = float(price.replace('Lakh', '').strip()) * 100000
    return price

In [3]:
# Function to convert JSON-like strings to dictionaries
def convert_to_dict(x):
    if isinstance(x, str):
        return json.loads(x.replace("'", '"').replace("None", "null"))
    return x


In [4]:
# funtion to extract from feature
def extract_feature(x):
    for i in df['new_car_feature']:
        feature=[]
        for j in i['top']:
            feature.append(j['value'])
    return x

In [ ]:
# Apply the function to the column
df['new_car_overview'] = df['new_car_overview'].apply(convert_to_dict)

# Extract the 'top' section and create a new DataFrame
top_data = df['new_car_overview'].apply(lambda x: {item['key']: item['value'] for item in x['top']})

# Convert the list of dictionaries to a DataFrame
overview_df = pd.DataFrame(top_data.to_list())
overview_df = overview_df.drop(['Year of Manufacture','RTO','Transmission','Ownership'],axis=1)
overview_df['SerialNumber'] = range(1, 1 + len(df))

overview_df['Registration Year'] = overview_df['Registration Year'].str.extract(r'(\d{4})')
overview_df['Seats'] = overview_df['Seats'].str.replace(r'\D', '', regex=True)
overview_df['Kms Driven'] = overview_df['Kms Driven'].str.replace(r'\D', '', regex=True)
overview_df['Engine Displacement'] = overview_df['Engine Displacement'].str.replace(r'\D', '', regex=True)
overview_df.fillna(method='ffill',inplace=True)
overview_df.replace(np.nan,0)

In [ ]:

df['new_car_feature'] = df['new_car_feature'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_feature(feature):
    flattened = {f"top_{i}": item['value'] for i, item in enumerate(feature.get('top', []))}
    for section in feature.get('data', []):
        for i, item in enumerate(section.get('list', [])):
            flattened[f"{section['heading']}_{i}"] = item['value']
    return flattened

# Apply the function to the column
flattened_feature = df['new_car_feature'].apply(flatten_feature)

# Create a DataFrame from the flattened data
feature_df = pd.DataFrame(flattened_feature.tolist())

cols = ['Comfort & Convenience_16','Interior_10','Safety_26','Entertainment & Communication_8',
'Comfort & Convenience_19','Comfort & Convenience_20', 'Exterior_13', 'Exterior_14', 'Safety_27',
'Safety_28', 'Safety_29', 'Safety_30', 'Comfort & Convenience_21',
'Interior_12', 'Exterior_15','Comfort & Convenience_22', 'Comfort & Convenience_23',
'Comfort & Convenience_24', 'Comfort & Convenience_25', 'Interior_13','Interior_14', 'Exterior_16', 'Exterior_17', 'Exterior_18',
'Exterior_19','Exterior_20', 'Exterior_21', 'Exterior_22', 'Safety_31', 'Safety_32',
'Entertainment & Communication_9', 'Entertainment & Communication_10',
'Entertainment & Communication_11', 'Comfort & Convenience_26',
'Comfort & Convenience_27','Comfort & Convenience_28', 'Comfort & Convenience_29',
'Comfort & Convenience_30', 'Safety_33', 'Safety_34', 'Safety_35',
'Safety_36', 'Safety_37', 'Entertainment & Communication_12',
'Comfort & Convenience_31','Interior_15', 'Entertainment & Communication_13',
'Entertainment & Communication_14', 'Entertainment & Communication_15',
'Entertainment & Communication_16', 'Entertainment & Communication_17',
'Entertainment & Communication_18', 'Entertainment & Communication_19',
'Entertainment & Communication_20', 'Safety_38','Safety_39', 'Safety_40', 'Safety_41', 'Safety_42', 'Safety_43',
'Safety_44', 'Exterior_23', 'Exterior_24', 'Exterior_25',
'Exterior_26','Comfort & Convenience_32', 'Comfort & Convenience_33',
'Comfort & Convenience_34']

feature_df.drop(cols, axis=1, inplace = True)
feature_df['SerialNumber'] = range(1, 1 + len(df))
feature_df.fillna(0,inplace=True)
feature_df.replace(np.nan,0)

In [59]:

df['new_car_specs'] = df['new_car_specs'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_specs(specs):
    flattened = {item['key']: item['value'] for item in specs.get('top', [])}
    for section in specs.get('data', []):
        for item in section.get('list', []):
            flattened[item['key']] = item['value']
    return flattened

# Apply the function to the column
flattened_specs = df['new_car_specs'].apply(flatten_specs)

specs_df = pd.DataFrame(flattened_specs.tolist())

cols = ['Engine','Max Power','Max Torque','Value Configuration','Fuel Suppy System','BoreX Stroke','Compression Ratio','Acceleration',
        'Wheel Base']
specs_df.drop(cols, axis=1,inplace=True)
specs_df['Mileage'] = specs_df['Mileage'].str.replace(r'\D', '', regex=True)
specs_df['Length'] = specs_df['Length'].str.replace(r'\D', '', regex=True)
specs_df['Width'] = specs_df['Width'].str.replace(r'\D', '', regex=True)
specs_df['Height'] = specs_df['Height'].str.replace(r'\D', '', regex=True)
specs_df['Front Tread'] = specs_df['Front Tread'].str.replace(r'\D', '', regex=True)
specs_df['Rear Tread'] = specs_df['Rear Tread'].str.replace(r'\D', '', regex=True)
specs_df['Kerb Weight'] = specs_df['Kerb Weight'].str.replace(r'\D', '', regex=True)
specs_df['Gear Box'] = specs_df['Gear Box'].str.replace(r'\D', '', regex=True)
specs_df['Turning Radius'] = specs_df['Turning Radius'].str.replace(r'\D', '', regex=True)
specs_df['Top Speed'] = specs_df['Top Speed'].str.replace(r'\D', '', regex=True)
specs_df['Cargo Volumn'] = specs_df['Cargo Volumn'].str.replace(r'\D', '', regex=True)
specs_df['Ground Clearance Unladen'] = specs_df['Ground Clearance Unladen'].str.replace(r'\D', '', regex=True)
specs_df['Torque'] = specs_df['Torque'].str.replace(r'\D', '', regex=True)
specs_df['SerialNumber'] = range(1, 1 + len(df))

specs_df['Turbo Charger'] = specs_df['Turbo Charger'].replace({'Yes': 1, 'No': 0})
specs_df['Super Charger'] = specs_df['Super Charger'].replace({'Yes': 1, 'No': 0})

specs_df.fillna(method='ffill', inplace=True)
specs_df.fillna(method='bfill', inplace=True)

C:\Users\Ragavan lingam\AppData\Local\Temp\ipykernel_5832\1960900201.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  specs_df.fillna(method='ffill', inplace=True)
C:\Users\Ragavan lingam\AppData\Local\Temp\ipykernel_5832\1960900201.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  specs_df.fillna(method='bfill', inplace=True)


In [53]:

dfs = [detail_df, overview_df, feature_df,specs_df]

Bangalore_df = pd.concat(dfs, ignore_index=True)
# Bangalore_df.to_csv('Bangalore_cars.csv', index=False)

In [61]:


# Merge detail_df and overview_df
merged_df = pd.merge(detail_df, overview_df, on='SerialNumber')

# Merge the result with feature_df
merged_df = pd.merge(merged_df, feature_df, on='SerialNumber')

# Finally, merge the result with specs_df
Bangalore_df = pd.merge(merged_df, specs_df, on='SerialNumber')


In [77]:
Bangalore_df.to_csv('Bangalore_cars.csv', index=False)

Chennai

In [78]:
df= pd.read_excel(r"C:\Users\Ragavan lingam\OneDrive\Desktop\MD112\Capstone\Car Dheko - Used Car Price Prediction\chennai_cars.xlsx")

df['new_car_detail'] = df['new_car_detail'].apply(lambda x: json.loads(x.replace("'", '"').replace("None", "1")))
detail_df=pd.DataFrame(df.new_car_detail.to_list())
detail_df = detail_df.drop(['priceActual','priceSaving','priceFixedText','trendingText','oem','it','owner'],axis=1)
detail_df['price'] = detail_df['price'].apply(convert_price)
detail_df['City'] = 'Chennai'
detail_df['SerialNumber'] = range(1, 1 + len(df))

In [79]:
# Apply the function to the column
df['new_car_overview'] = df['new_car_overview'].apply(convert_to_dict)

# Extract the 'top' section and create a new DataFrame
top_data = df['new_car_overview'].apply(lambda x: {item['key']: item['value'] for item in x['top']})

# Convert the list of dictionaries to a DataFrame
overview_df = pd.DataFrame(top_data.to_list())
overview_df = overview_df.drop(['Year of Manufacture','RTO','Transmission','Ownership'],axis=1)
overview_df['SerialNumber'] = range(1, 1 + len(df))

overview_df['Registration Year'] = overview_df['Registration Year'].str.extract(r'(\d{4})')
overview_df['Seats'] = overview_df['Seats'].str.replace(r'\D', '', regex=True)
overview_df['Kms Driven'] = overview_df['Kms Driven'].str.replace(r'\D', '', regex=True)
overview_df['Engine Displacement'] = overview_df['Engine Displacement'].str.replace(r'\D', '', regex=True)

In [80]:

df['new_car_feature'] = df['new_car_feature'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_feature(feature):
    flattened = {f"top_{i}": item['value'] for i, item in enumerate(feature.get('top', []))}
    for section in feature.get('data', []):
        for i, item in enumerate(section.get('list', [])):
            flattened[f"{section['heading']}_{i}"] = item['value']
    return flattened

# Apply the function to the column
flattened_feature = df['new_car_feature'].apply(flatten_feature)

# Create a DataFrame from the flattened data
feature_df = pd.DataFrame(flattened_feature.tolist())

cols = ['Comfort & Convenience_16','Interior_10','Safety_26','Entertainment & Communication_8',
'Comfort & Convenience_19','Comfort & Convenience_20', 'Exterior_13', 'Exterior_14', 'Safety_27',
'Safety_28', 'Safety_29', 'Safety_30', 'Comfort & Convenience_21',
'Interior_12', 'Exterior_15','Comfort & Convenience_22', 'Comfort & Convenience_23',
'Comfort & Convenience_24', 'Comfort & Convenience_25', 'Interior_13','Interior_14', 'Exterior_16', 'Exterior_17', 'Exterior_18',
'Exterior_19','Exterior_20', 'Exterior_21', 'Exterior_22', 'Safety_31', 'Safety_32',
'Entertainment & Communication_9', 'Entertainment & Communication_10',
'Entertainment & Communication_11', 'Comfort & Convenience_26',
'Comfort & Convenience_27','Comfort & Convenience_28', 'Comfort & Convenience_29',
'Comfort & Convenience_30', 'Safety_33', 'Safety_34', 'Safety_35',
'Safety_36', 'Safety_37', 'Entertainment & Communication_12',
'Comfort & Convenience_31','Interior_15', 'Entertainment & Communication_13',
'Entertainment & Communication_14', 'Entertainment & Communication_15',
'Entertainment & Communication_16', 'Entertainment & Communication_17',
'Entertainment & Communication_18','Safety_38','Safety_39', 'Safety_40', 'Safety_41', 'Safety_42', 'Safety_43',
'Safety_44', 'Exterior_23', 'Exterior_24', 'Exterior_25',
'Comfort & Convenience_32', 'Comfort & Convenience_33',
'Comfort & Convenience_34']

feature_df.drop(cols, axis=1, inplace = True)
feature_df['SerialNumber'] = range(1, 1 + len(df))
feature_df.fillna(0,inplace=True)

In [81]:

df['new_car_specs'] = df['new_car_specs'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_specs(specs):
    flattened = {item['key']: item['value'] for item in specs.get('top', [])}
    for section in specs.get('data', []):
        for item in section.get('list', []):
            flattened[item['key']] = item['value']
    return flattened

# Apply the function to the column
flattened_specs = df['new_car_specs'].apply(flatten_specs)

specs_df = pd.DataFrame(flattened_specs.tolist())


In [82]:
cols = ['Engine','Max Power','Max Torque','Value Configuration','Fuel Suppy System','BoreX Stroke','Compression Ratio','Acceleration',
        'Wheel Base']
specs_df.drop(cols, axis=1,inplace=True)
specs_df['Mileage'] = specs_df['Mileage'].str.replace(r'\D', '', regex=True)
specs_df['Length'] = specs_df['Length'].str.replace(r'\D', '', regex=True)
specs_df['Width'] = specs_df['Width'].str.replace(r'\D', '', regex=True)
specs_df['Height'] = specs_df['Height'].str.replace(r'\D', '', regex=True)
specs_df['Front Tread'] = specs_df['Front Tread'].str.replace(r'\D', '', regex=True)
specs_df['Rear Tread'] = specs_df['Rear Tread'].str.replace(r'\D', '', regex=True)
specs_df['Kerb Weight'] = specs_df['Kerb Weight'].str.replace(r'\D', '', regex=True)
specs_df['Gear Box'] = specs_df['Gear Box'].str.replace(r'\D', '', regex=True)
specs_df['Turning Radius'] = specs_df['Turning Radius'].str.replace(r'\D', '', regex=True)
specs_df['Top Speed'] = specs_df['Top Speed'].str.replace(r'\D', '', regex=True)
specs_df['Cargo Volumn'] = specs_df['Cargo Volumn'].str.replace(r'\D', '', regex=True)
specs_df['Ground Clearance Unladen'] = specs_df['Ground Clearance Unladen'].str.replace(r'\D', '', regex=True)
specs_df['Torque'] = specs_df['Torque'].str.replace(r'\D', '', regex=True)
specs_df['SerialNumber'] = range(1, 1 + len(df))

specs_df['Turbo Charger'] = specs_df['Turbo Charger'].replace({'Yes': 1, 'No': 0})
specs_df['Super Charger'] = specs_df['Super Charger'].replace({'Yes': 1, 'No': 0})

specs_df.fillna(method='ffill', inplace=True)
specs_df.replace(np.nan,0)

C:\Users\Ragavan lingam\AppData\Local\Temp\ipykernel_5832\2564426943.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  specs_df.fillna(method='ffill', inplace=True)


,Torque,Wheel Size,Seats,Color,Engine Type,Displacement,No of Cylinder,Values per Cylinder,Turbo Charger,Length,...,Cargo Volumn,Mileage,Super Charger,Front Tread,Rear Tread,Gross Weight,Turning Radius,Top Speed,Ground Clearance Unladen,SerialNumber
0,172,16,5,Black,Smartstream G1.0 T - GDi,998,3.0,4.0,1,3995,...,392,0,0,0,0,0,0,0,0,1
1,101,16,7,Grey,In-Line Engine,1196,4.0,4.0,0,3675,...,540,1537,0,1280,1290,1540kg,45,145,0,2
2,160,16,5,Others,HRA0 1.0 TURBO PETROL,999,3.0,4.0,1,3994,...,336,20,0,1280,1290,1540kg,50,145,0,3
3,9904,16,5,Silver,IRDE2 Engine,1086,4.0,3.0,0,3585,...,225,1981,0,1400,1385,1540kg,47,165,0,4
4,110,15,5,Others,i-VTEC Petrol Engine,1199,4.0,4.0,0,3955,...,354,187,0,1400,1385,1540kg,51,172,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,113,17,5,Blue,VTVT Engine,1197,4.0,4.0,0,3840,...,268,22,0,1530,1530,1315kg,48,242,184,1415
1415,200,16,5,Orange,dCi THP Diesel Engine,1461,4.0,4.0,1,4315,...,475,1987,0,1560,1567,1777kg,52,156,184,1416
1416,170,16,5,White,Revotron 1.2L Turbocharged engine,1198,3.0,4.0,1,3994,...,350,17,0,1540,1530,1777kg,51,15419,209,1417
1417,340,18,5,Grey,Common Rail Direct Injection,1968,4.0,4.0,1,4486,...,615,1665,0,1578,1568,2250kg,575,15419,209,1418


In [83]:

# Merge detail_df and overview_df
merged_df = pd.merge(detail_df, overview_df, on='SerialNumber')

# Merge the result with feature_df
merged_df = pd.merge(merged_df, feature_df, on='SerialNumber')

# Finally, merge the result with specs_df
Chennai_df = pd.merge(merged_df, specs_df, on='SerialNumber')


In [86]:
# Chennai_df.drop(['SerialNumber'], axis=1)
Chennai_df.to_csv('Chennai_car.csv', index=False)

Delhi

In [87]:
df=pd.read_excel(r"C:\Users\Ragavan lingam\OneDrive\Desktop\MD112\Capstone\Car Dheko - Used Car Price Prediction\delhi_cars.xlsx")
df['new_car_detail'] = df['new_car_detail'].apply(lambda x: json.loads(x.replace("'", '"').replace("None", "1")))
detail_df=pd.DataFrame(df.new_car_detail.to_list())
detail_df = detail_df.drop(['priceActual','priceSaving','priceFixedText','trendingText','oem','it','owner'],axis=1)
detail_df['price'] = detail_df['price'].apply(convert_price)
detail_df['City'] = 'Delhi'
detail_df['SerialNumber'] = range(1, 1 + len(df))

In [88]:
# Apply the function to the column
df['new_car_overview'] = df['new_car_overview'].apply(convert_to_dict)

# Extract the 'top' section and create a new DataFrame
top_data = df['new_car_overview'].apply(lambda x: {item['key']: item['value'] for item in x['top']})

# Convert the list of dictionaries to a DataFrame
overview_df = pd.DataFrame(top_data.to_list())
overview_df = overview_df.drop(['Year of Manufacture','RTO','Transmission','Ownership'],axis=1)
overview_df['SerialNumber'] = range(1, 1 + len(df))

overview_df['Registration Year'] = overview_df['Registration Year'].str.extract(r'(\d{4})')
overview_df['Seats'] = overview_df['Seats'].str.replace(r'\D', '', regex=True)
overview_df['Kms Driven'] = overview_df['Kms Driven'].str.replace(r'\D', '', regex=True)
overview_df['Engine Displacement'] = overview_df['Engine Displacement'].str.replace(r'\D', '', regex=True)

In [ ]:

df['new_car_feature'] = df['new_car_feature'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_feature(feature):
    flattened = {f"top_{i}": item['value'] for i, item in enumerate(feature.get('top', []))}
    for section in feature.get('data', []):
        for i, item in enumerate(section.get('list', [])):
            flattened[f"{section['heading']}_{i}"] = item['value']
    return flattened

# Apply the function to the column
flattened_feature = df['new_car_feature'].apply(flatten_feature)

# Create a DataFrame from the flattened data
feature_df = pd.DataFrame(flattened_feature.tolist())

cols = ['Comfort & Convenience_16','Interior_10','Safety_26','Entertainment & Communication_8',
'Comfort & Convenience_19','Comfort & Convenience_20', 'Exterior_13', 'Exterior_14', 'Safety_27',
'Safety_28', 'Safety_29', 'Safety_30', 'Comfort & Convenience_21',
'Interior_12', 'Exterior_15','Comfort & Convenience_22', 'Comfort & Convenience_23',
'Comfort & Convenience_24', 'Comfort & Convenience_25', 'Interior_13','Interior_14', 'Exterior_16', 'Exterior_17', 'Exterior_18',
'Exterior_19','Exterior_20', 'Exterior_21', 'Exterior_22', 'Safety_31', 'Safety_32',
'Entertainment & Communication_9', 'Entertainment & Communication_10',
'Entertainment & Communication_11', 'Comfort & Convenience_26',
'Comfort & Convenience_27','Comfort & Convenience_28', 'Comfort & Convenience_29',
'Comfort & Convenience_30', 'Safety_33', 'Safety_34', 'Safety_35',
'Safety_36', 'Safety_37', 'Entertainment & Communication_12',
'Comfort & Convenience_31','Interior_15', 'Entertainment & Communication_13',
'Entertainment & Communication_14', 'Entertainment & Communication_15',
'Entertainment & Communication_16', 'Entertainment & Communication_17',
'Entertainment & Communication_18', 'Entertainment & Communication_19',
'Entertainment & Communication_20', 'Safety_38','Safety_39', 'Safety_40', 'Safety_41', 'Safety_42', 'Safety_43',
'Safety_44', 'Exterior_23', 'Exterior_24', 'Exterior_25','Comfort & Convenience_32', 'Comfort & Convenience_33',
'Comfort & Convenience_34']

feature_df['SerialNumber'] = range(1, 1 + len(df))

feature_df.drop(cols, axis=1, inplace = True)
feature_df.fillna(0,inplace=True)
feature_df.replace(np.nan,0)

In [ ]:

df['new_car_specs'] = df['new_car_specs'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_specs(specs):
    flattened = {item['key']: item['value'] for item in specs.get('top', [])}
    for section in specs.get('data', []):
        for item in section.get('list', []):
            flattened[item['key']] = item['value']
    return flattened

# Apply the function to the column
flattened_specs = df['new_car_specs'].apply(flatten_specs)

specs_df = pd.DataFrame(flattened_specs.tolist())

cols = ['Engine','Max Power','Max Torque','Value Configuration','Fuel Suppy System','BoreX Stroke','Compression Ratio','Acceleration',
        'Wheel Base']
specs_df.drop(cols, axis=1,inplace=True)
specs_df['Mileage'] = specs_df['Mileage'].str.replace(r'\D', '', regex=True)
specs_df['Length'] = specs_df['Length'].str.replace(r'\D', '', regex=True)
specs_df['Width'] = specs_df['Width'].str.replace(r'\D', '', regex=True)
specs_df['Height'] = specs_df['Height'].str.replace(r'\D', '', regex=True)
specs_df['Front Tread'] = specs_df['Front Tread'].str.replace(r'\D', '', regex=True)
specs_df['Rear Tread'] = specs_df['Rear Tread'].str.replace(r'\D', '', regex=True)
specs_df['Kerb Weight'] = specs_df['Kerb Weight'].str.replace(r'\D', '', regex=True)
specs_df['Gear Box'] = specs_df['Gear Box'].str.replace(r'\D', '', regex=True)
specs_df['Turning Radius'] = specs_df['Turning Radius'].str.replace(r'\D', '', regex=True)
specs_df['Top Speed'] = specs_df['Top Speed'].str.replace(r'\D', '', regex=True)
specs_df['Cargo Volumn'] = specs_df['Cargo Volumn'].str.replace(r'\D', '', regex=True)
specs_df['Ground Clearance Unladen'] = specs_df['Ground Clearance Unladen'].str.replace(r'\D', '', regex=True)
specs_df['Torque'] = specs_df['Torque'].str.replace(r'\D', '', regex=True)
specs_df['SerialNumber'] = range(1, 1 + len(df))


specs_df['Turbo Charger'] = specs_df['Turbo Charger'].replace({'Yes': 1, 'No': 0})
specs_df['Super Charger'] = specs_df['Super Charger'].replace({'Yes': 1, 'No': 0})

specs_df.fillna(method='ffill', inplace=True)
specs_df.replace(np.nan,0)


In [91]:
# Merge detail_df and overview_df
merged_df = pd.merge(detail_df, overview_df, on='SerialNumber')

# Merge the result with feature_df
merged_df = pd.merge(merged_df, feature_df, on='SerialNumber')

# Finally, merge the result with specs_df
Delhi_df = pd.merge(merged_df, specs_df, on='SerialNumber')

In [92]:
Delhi_df.drop(['SerialNumber'], axis=1)
Delhi_df.to_csv('Delhi_cars.csv', index=False)

Hyderabad

In [99]:
df=pd.read_excel(r"C:\Users\Ragavan lingam\OneDrive\Desktop\MD112\Capstone\Car Dheko - Used Car Price Prediction\hyderabad_cars.xlsx")
df['new_car_detail'] = df['new_car_detail'].apply(lambda x: json.loads(x.replace("'", '"').replace("None", "1")))
detail_df=pd.DataFrame(df.new_car_detail.to_list())
detail_df = detail_df.drop(['priceActual','priceSaving','priceFixedText','trendingText','oem','it','owner'],axis=1)
detail_df['price'] = detail_df['price'].apply(convert_price)
detail_df['City'] = 'Hyderabad'
detail_df['SerialNumber'] = range(1, 1 + len(df))

In [100]:
# Apply the function to the column
df['new_car_overview'] = df['new_car_overview'].apply(convert_to_dict)

# Extract the 'top' section and create a new DataFrame
top_data = df['new_car_overview'].apply(lambda x: {item['key']: item['value'] for item in x['top']})

# Convert the list of dictionaries to a DataFrame
overview_df = pd.DataFrame(top_data.to_list())
overview_df = overview_df.drop(['Year of Manufacture','RTO','Transmission','Ownership'],axis=1)
overview_df['SerialNumber'] = range(1, 1 + len(df))

overview_df['Registration Year'] = overview_df['Registration Year'].str.extract(r'(\d{4})')
overview_df['Seats'] = overview_df['Seats'].str.replace(r'\D', '', regex=True)
overview_df['Kms Driven'] = overview_df['Kms Driven'].str.replace(r'\D', '', regex=True)
overview_df['Engine Displacement'] = overview_df['Engine Displacement'].str.replace(r'\D', '', regex=True)

In [ ]:

df['new_car_feature'] = df['new_car_feature'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_feature(feature):
    flattened = {f"top_{i}": item['value'] for i, item in enumerate(feature.get('top', []))}
    for section in feature.get('data', []):
        for i, item in enumerate(section.get('list', [])):
            flattened[f"{section['heading']}_{i}"] = item['value']
    return flattened

# Apply the function to the column
flattened_feature = df['new_car_feature'].apply(flatten_feature)

# Create a DataFrame from the flattened data
feature_df = pd.DataFrame(flattened_feature.tolist())

cols = ['Comfort & Convenience_16','Interior_10','Safety_26','Entertainment & Communication_8',
'Comfort & Convenience_19','Comfort & Convenience_20', 'Exterior_13', 'Exterior_14', 'Safety_27',
'Safety_28', 'Safety_29', 'Safety_30', 'Comfort & Convenience_21',
'Interior_12', 'Exterior_15','Comfort & Convenience_22', 'Comfort & Convenience_23',
'Comfort & Convenience_24', 'Comfort & Convenience_25', 'Interior_13','Interior_14', 'Exterior_16', 'Exterior_17', 'Exterior_18',
'Exterior_19','Exterior_20', 'Exterior_21', 'Exterior_22', 'Safety_31', 'Safety_32',
'Entertainment & Communication_9', 'Entertainment & Communication_10',
'Entertainment & Communication_11', 'Comfort & Convenience_26',
'Comfort & Convenience_27','Comfort & Convenience_28', 'Comfort & Convenience_29',
'Comfort & Convenience_30', 'Safety_33', 'Safety_34', 'Safety_35',
'Safety_36', 'Safety_37', 'Entertainment & Communication_12',
'Comfort & Convenience_31','Interior_15', 'Entertainment & Communication_13',
'Entertainment & Communication_14', 'Entertainment & Communication_15',
'Entertainment & Communication_16', 'Entertainment & Communication_17',
'Entertainment & Communication_18', 'Safety_38','Safety_39', 'Exterior_23', 'Exterior_24']

feature_df['SerialNumber'] = range(1, 1 + len(df))

feature_df.drop(cols, axis=1, inplace = True)
feature_df.fillna(0,inplace=True)
feature_df.replace(np.nan,0)

In [104]:

df['new_car_specs'] = df['new_car_specs'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_specs(specs):
    flattened = {item['key']: item['value'] for item in specs.get('top', [])}
    for section in specs.get('data', []):
        for item in section.get('list', []):
            flattened[item['key']] = item['value']
    return flattened

# Apply the function to the column
flattened_specs = df['new_car_specs'].apply(flatten_specs)

specs_df = pd.DataFrame(flattened_specs.tolist())

cols = ['Engine','Max Power','Max Torque','Value Configuration','Fuel Suppy System','BoreX Stroke','Compression Ratio','Acceleration',
        'Wheel Base']
specs_df.drop(cols, axis=1,inplace=True)
specs_df['Mileage'] = specs_df['Mileage'].str.replace(r'\D', '', regex=True)
specs_df['Length'] = specs_df['Length'].str.replace(r'\D', '', regex=True)
specs_df['Width'] = specs_df['Width'].str.replace(r'\D', '', regex=True)
specs_df['Height'] = specs_df['Height'].str.replace(r'\D', '', regex=True)
specs_df['Front Tread'] = specs_df['Front Tread'].str.replace(r'\D', '', regex=True)
specs_df['Rear Tread'] = specs_df['Rear Tread'].str.replace(r'\D', '', regex=True)
specs_df['Kerb Weight'] = specs_df['Kerb Weight'].str.replace(r'\D', '', regex=True)
specs_df['Gear Box'] = specs_df['Gear Box'].str.replace(r'\D', '', regex=True)
specs_df['Turning Radius'] = specs_df['Turning Radius'].str.replace(r'\D', '', regex=True)
specs_df['Top Speed'] = specs_df['Top Speed'].str.replace(r'\D', '', regex=True)
specs_df['Cargo Volumn'] = specs_df['Cargo Volumn'].str.replace(r'\D', '', regex=True)
specs_df['Ground Clearance Unladen'] = specs_df['Ground Clearance Unladen'].str.replace(r'\D', '', regex=True)
specs_df['Torque'] = specs_df['Torque'].str.replace(r'\D', '', regex=True)
specs_df['SerialNumber'] = range(1, 1 + len(df))

specs_df['Turbo Charger'] = specs_df['Turbo Charger'].replace({'Yes': 1, 'No': 0})
specs_df['Super Charger'] = specs_df['Super Charger'].replace({'Yes': 1, 'No': 0})

specs_df.fillna(method='ffill', inplace=True)
specs_df.fillna(method='bfill', inplace=True)

C:\Users\Ragavan lingam\AppData\Local\Temp\ipykernel_5832\1960900201.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  specs_df.fillna(method='ffill', inplace=True)
C:\Users\Ragavan lingam\AppData\Local\Temp\ipykernel_5832\1960900201.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  specs_df.fillna(method='bfill', inplace=True)


In [106]:
# Merge detail_df and overview_df
merged_df = pd.merge(detail_df, overview_df, on='SerialNumber')

# Merge the result with feature_df
merged_df = pd.merge(merged_df, feature_df, on='SerialNumber')

# Finally, merge the result with specs_df
Hyderabad_df = pd.merge(merged_df, specs_df, on='SerialNumber')

In [107]:
Hyderabad_df.drop(['SerialNumber'], axis=1)
Hyderabad_df.to_csv('Hyderabad_cars.csv', index=False)

Jaipur

In [111]:
df = pd.read_excel(r"C:\Users\Ragavan lingam\OneDrive\Desktop\MD112\Capstone\Car Dheko - Used Car Price Prediction\jaipur_cars.xlsx")
df['new_car_detail'] = df['new_car_detail'].apply(lambda x: json.loads(x.replace("'", '"').replace("None", "1")))
detail_df=pd.DataFrame(df.new_car_detail.to_list())
detail_df = detail_df.drop(['priceActual','priceSaving','priceFixedText','trendingText','oem','it','owner'],axis=1)
detail_df['price'] = detail_df['price'].apply(convert_price)
detail_df['City'] = 'Bangalore'
detail_df['SerialNumber'] = range(1, 1 + len(df))

In [ ]:
# Apply the function to the column
df['new_car_overview'] = df['new_car_overview'].apply(convert_to_dict)

# Extract the 'top' section and create a new DataFrame
top_data = df['new_car_overview'].apply(lambda x: {item['key']: item['value'] for item in x['top']})

# Convert the list of dictionaries to a DataFrame
overview_df = pd.DataFrame(top_data.to_list())
overview_df = overview_df.drop(['Year of Manufacture','RTO','Transmission','Ownership'],axis=1)
overview_df['SerialNumber'] = range(1, 1 + len(df))

overview_df['Registration Year'] = overview_df['Registration Year'].str.extract(r'(\d{4})')
overview_df['Seats'] = overview_df['Seats'].str.replace(r'\D', '', regex=True)
overview_df['Kms Driven'] = overview_df['Kms Driven'].str.replace(r'\D', '', regex=True)
overview_df['Engine Displacement'] = overview_df['Engine Displacement'].str.replace(r'\D', '', regex=True)
overview_df.fillna(method='ffill',inplace=True)
overview_df.replace(np.nan,0)

In [ ]:

df['new_car_feature'] = df['new_car_feature'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_feature(feature):
    flattened = {f"top_{i}": item['value'] for i, item in enumerate(feature.get('top', []))}
    for section in feature.get('data', []):
        for i, item in enumerate(section.get('list', [])):
            flattened[f"{section['heading']}_{i}"] = item['value']
    return flattened

# Apply the function to the column
flattened_feature = df['new_car_feature'].apply(flatten_feature)

# Create a DataFrame from the flattened data
feature_df = pd.DataFrame(flattened_feature.tolist())

cols = ['Comfort & Convenience_16','Interior_10','Safety_26','Entertainment & Communication_8',
'Comfort & Convenience_19','Comfort & Convenience_20', 'Exterior_13', 'Exterior_14', 'Safety_27',
'Safety_28', 'Safety_29', 'Safety_30', 'Comfort & Convenience_21',
'Interior_12', 'Exterior_15','Comfort & Convenience_22', 'Comfort & Convenience_23',
'Comfort & Convenience_24', 'Comfort & Convenience_25', 'Interior_13','Interior_14', 'Exterior_16', 'Exterior_17', 'Exterior_18',
'Exterior_19','Exterior_20', 'Exterior_21', 'Exterior_22', 'Safety_31', 'Safety_32',
'Entertainment & Communication_9', 'Entertainment & Communication_10',
'Entertainment & Communication_11', 'Comfort & Convenience_26',
'Comfort & Convenience_27','Comfort & Convenience_28', 'Comfort & Convenience_29',
'Comfort & Convenience_30', 'Safety_33', 'Safety_34', 'Safety_35',
'Safety_36', 'Safety_37', 'Entertainment & Communication_12',
'Comfort & Convenience_31','Interior_15', 'Entertainment & Communication_13',
'Entertainment & Communication_14', 'Safety_38','Safety_39', 'Safety_40', 'Safety_41','Exterior_23', 'Exterior_24', 'Exterior_25',
'Comfort & Convenience_32']

feature_df.drop(cols, axis=1, inplace = True)
feature_df['SerialNumber'] = range(1, 1 + len(df))
feature_df.fillna(0,inplace=True)
feature_df.replace(np.nan,0)

In [115]:

df['new_car_specs'] = df['new_car_specs'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_specs(specs):
    flattened = {item['key']: item['value'] for item in specs.get('top', [])}
    for section in specs.get('data', []):
        for item in section.get('list', []):
            flattened[item['key']] = item['value']
    return flattened

# Apply the function to the column
flattened_specs = df['new_car_specs'].apply(flatten_specs)

specs_df = pd.DataFrame(flattened_specs.tolist())

cols = ['Engine','Max Power','Max Torque','Value Configuration','Fuel Suppy System','BoreX Stroke','Compression Ratio','Acceleration',
        'Wheel Base']
specs_df.drop(cols, axis=1,inplace=True)
specs_df['Mileage'] = specs_df['Mileage'].str.replace(r'\D', '', regex=True)
specs_df['Length'] = specs_df['Length'].str.replace(r'\D', '', regex=True)
specs_df['Width'] = specs_df['Width'].str.replace(r'\D', '', regex=True)
specs_df['Height'] = specs_df['Height'].str.replace(r'\D', '', regex=True)
specs_df['Front Tread'] = specs_df['Front Tread'].str.replace(r'\D', '', regex=True)
specs_df['Rear Tread'] = specs_df['Rear Tread'].str.replace(r'\D', '', regex=True)
specs_df['Kerb Weight'] = specs_df['Kerb Weight'].str.replace(r'\D', '', regex=True)
specs_df['Gear Box'] = specs_df['Gear Box'].str.replace(r'\D', '', regex=True)
specs_df['Turning Radius'] = specs_df['Turning Radius'].str.replace(r'\D', '', regex=True)
specs_df['Top Speed'] = specs_df['Top Speed'].str.replace(r'\D', '', regex=True)
specs_df['Cargo Volumn'] = specs_df['Cargo Volumn'].str.replace(r'\D', '', regex=True)
specs_df['Ground Clearance Unladen'] = specs_df['Ground Clearance Unladen'].str.replace(r'\D', '', regex=True)
specs_df['Torque'] = specs_df['Torque'].str.replace(r'\D', '', regex=True)
specs_df['SerialNumber'] = range(1, 1 + len(df))

specs_df['Turbo Charger'] = specs_df['Turbo Charger'].replace({'Yes': 1, 'No': 0})
specs_df['Super Charger'] = specs_df['Super Charger'].replace({'Yes': 1, 'No': 0})

specs_df.fillna(method='ffill', inplace=True)
specs_df.fillna(method='bfill', inplace=True)

C:\Users\Ragavan lingam\AppData\Local\Temp\ipykernel_5832\1960900201.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  specs_df.fillna(method='ffill', inplace=True)
C:\Users\Ragavan lingam\AppData\Local\Temp\ipykernel_5832\1960900201.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  specs_df.fillna(method='bfill', inplace=True)


In [116]:
# Merge detail_df and overview_df
merged_df = pd.merge(detail_df, overview_df, on='SerialNumber')

# Merge the result with feature_df
merged_df = pd.merge(merged_df, feature_df, on='SerialNumber')

# Finally, merge the result with specs_df
Jaipur_df = pd.merge(merged_df, specs_df, on='SerialNumber')

Jaipur_df.drop(['SerialNumber'], axis=1)
Jaipur_df.to_csv('Jaipur_cars.csv', index=False)


Kolkata

In [119]:
df=pd.read_excel(r"C:\Users\Ragavan lingam\OneDrive\Desktop\MD112\Capstone\Car Dheko - Used Car Price Prediction\kolkata_cars.xlsx")
df['new_car_detail'] = df['new_car_detail'].apply(lambda x: json.loads(x.replace("'", '"').replace("None", "1")))
detail_df=pd.DataFrame(df.new_car_detail.to_list())
detail_df = detail_df.drop(['priceActual','priceSaving','priceFixedText','trendingText','oem','it','owner'],axis=1)
detail_df['price'] = detail_df['price'].apply(convert_price)
detail_df['City'] = 'Bangalore'
detail_df['SerialNumber'] = range(1, 1 + len(df))

In [ ]:
# Apply the function to the column
df['new_car_overview'] = df['new_car_overview'].apply(convert_to_dict)

# Extract the 'top' section and create a new DataFrame
top_data = df['new_car_overview'].apply(lambda x: {item['key']: item['value'] for item in x['top']})

# Convert the list of dictionaries to a DataFrame
overview_df = pd.DataFrame(top_data.to_list())
overview_df = overview_df.drop(['Year of Manufacture','RTO','Transmission','Ownership'],axis=1)
overview_df['SerialNumber'] = range(1, 1 + len(df))

overview_df['Registration Year'] = overview_df['Registration Year'].str.extract(r'(\d{4})')
overview_df['Seats'] = overview_df['Seats'].str.replace(r'\D', '', regex=True)
overview_df['Kms Driven'] = overview_df['Kms Driven'].str.replace(r'\D', '', regex=True)
overview_df['Engine Displacement'] = overview_df['Engine Displacement'].str.replace(r'\D', '', regex=True)
overview_df.fillna(method='ffill',inplace=True)
overview_df.replace(np.nan,0)

In [ ]:

df['new_car_feature'] = df['new_car_feature'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_feature(feature):
    flattened = {f"top_{i}": item['value'] for i, item in enumerate(feature.get('top', []))}
    for section in feature.get('data', []):
        for i, item in enumerate(section.get('list', [])):
            flattened[f"{section['heading']}_{i}"] = item['value']
    return flattened

# Apply the function to the column
flattened_feature = df['new_car_feature'].apply(flatten_feature)

# Create a DataFrame from the flattened data
feature_df = pd.DataFrame(flattened_feature.tolist())

cols = ['Comfort & Convenience_16','Interior_10','Safety_26','Entertainment & Communication_8',
'Comfort & Convenience_19','Comfort & Convenience_20', 'Exterior_13', 'Exterior_14', 'Safety_27',
'Safety_28', 'Safety_29', 'Safety_30', 'Comfort & Convenience_21',
'Interior_12', 'Exterior_15','Comfort & Convenience_22', 'Comfort & Convenience_23',
'Comfort & Convenience_24', 'Comfort & Convenience_25', 'Interior_13','Interior_14', 'Exterior_16', 'Exterior_17', 'Exterior_18',
'Exterior_19','Exterior_20', 'Exterior_21', 'Exterior_22', 'Safety_31', 'Safety_32',
'Entertainment & Communication_9', 'Entertainment & Communication_10',
'Entertainment & Communication_11', 'Comfort & Convenience_26',
'Comfort & Convenience_27','Comfort & Convenience_28', 'Comfort & Convenience_29',
'Comfort & Convenience_30', 'Safety_33', 'Safety_34', 'Safety_35',
'Safety_36', 'Safety_37', 'Entertainment & Communication_12',
'Comfort & Convenience_31','Interior_15', 'Entertainment & Communication_13',
'Entertainment & Communication_14', 'Entertainment & Communication_15',
'Entertainment & Communication_16', 'Entertainment & Communication_17',
'Entertainment & Communication_18', 'Safety_38','Safety_39', 'Safety_40', 'Safety_41', 'Safety_42', 'Safety_43',
'Safety_44', 'Exterior_23', 'Exterior_24', 'Exterior_25',
'Comfort & Convenience_32', 'Comfort & Convenience_33']

feature_df.drop(cols, axis=1, inplace = True)
feature_df['SerialNumber'] = range(1, 1 + len(df))
feature_df.fillna(0,inplace=True)
feature_df.replace(np.nan,0)

In [123]:

df['new_car_specs'] = df['new_car_specs'].apply(convert_to_dict)

# Function to flatten the nested structure
def flatten_specs(specs):
    flattened = {item['key']: item['value'] for item in specs.get('top', [])}
    for section in specs.get('data', []):
        for item in section.get('list', []):
            flattened[item['key']] = item['value']
    return flattened

# Apply the function to the column
flattened_specs = df['new_car_specs'].apply(flatten_specs)

specs_df = pd.DataFrame(flattened_specs.tolist())

cols = ['Engine','Max Power','Max Torque','Value Configuration','Fuel Suppy System','BoreX Stroke','Compression Ratio','Acceleration',
        'Wheel Base']
specs_df.drop(cols, axis=1,inplace=True)
specs_df['Mileage'] = specs_df['Mileage'].str.replace(r'\D', '', regex=True)
specs_df['Length'] = specs_df['Length'].str.replace(r'\D', '', regex=True)
specs_df['Width'] = specs_df['Width'].str.replace(r'\D', '', regex=True)
specs_df['Height'] = specs_df['Height'].str.replace(r'\D', '', regex=True)
specs_df['Front Tread'] = specs_df['Front Tread'].str.replace(r'\D', '', regex=True)
specs_df['Rear Tread'] = specs_df['Rear Tread'].str.replace(r'\D', '', regex=True)
specs_df['Kerb Weight'] = specs_df['Kerb Weight'].str.replace(r'\D', '', regex=True)
specs_df['Gear Box'] = specs_df['Gear Box'].str.replace(r'\D', '', regex=True)
specs_df['Turning Radius'] = specs_df['Turning Radius'].str.replace(r'\D', '', regex=True)
specs_df['Top Speed'] = specs_df['Top Speed'].str.replace(r'\D', '', regex=True)
specs_df['Cargo Volumn'] = specs_df['Cargo Volumn'].str.replace(r'\D', '', regex=True)
specs_df['Ground Clearance Unladen'] = specs_df['Ground Clearance Unladen'].str.replace(r'\D', '', regex=True)
specs_df['Torque'] = specs_df['Torque'].str.replace(r'\D', '', regex=True)
specs_df['SerialNumber'] = range(1, 1 + len(df))

specs_df['Turbo Charger'] = specs_df['Turbo Charger'].replace({'Yes': 1, 'No': 0})
specs_df['Super Charger'] = specs_df['Super Charger'].replace({'Yes': 1, 'No': 0})

specs_df.fillna(method='ffill', inplace=True)
specs_df.fillna(method='bfill', inplace=True)

C:\Users\Ragavan lingam\AppData\Local\Temp\ipykernel_5832\1960900201.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  specs_df.fillna(method='ffill', inplace=True)
C:\Users\Ragavan lingam\AppData\Local\Temp\ipykernel_5832\1960900201.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  specs_df.fillna(method='bfill', inplace=True)


In [124]:
# Merge detail_df and overview_df
merged_df = pd.merge(detail_df, overview_df, on='SerialNumber')

# Merge the result with feature_df
merged_df = pd.merge(merged_df, feature_df, on='SerialNumber')

# Finally, merge the result with specs_df
Kolkata_df = pd.merge(merged_df, specs_df, on='SerialNumber')

Kolkata_df.drop(['SerialNumber'], axis=1)
Kolkata_df.to_csv('Kolkata_cars.csv', index=False)